In [ ]:
!pip install albumentations --user
!pip install ultralytics --user
!pip install flask --user
!pip install scikit-learn --user

In [1]:
# Libraries for Data Preprocessing
import os
import cv2
import albumentations as A
import numpy as np

#Libraries for data splitting
import shutil
from sklearn.model_selection import train_test_split

# Libraries for ML Model
from ultralytics import YOLO

# Libraries for Python Web App
from flask import Flask, request, render_template, send_file

In [ ]:
# Function for data splitting
def custom_train_test_split(dataset_directory, test_ratio=0.2, random_seed=42, min_samples_per_class=2):
    
    class_directories = [d for d in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, d))]
    
    train_directory = os.path.join(dataset_directory, 'train')
    test_directory = os.path.join(dataset_directory, 'test')
    os.makedirs(train_directory, exist_ok=True)
    os.makedirs(test_directory, exist_ok=True)

    for class_dir in class_directories:
        class_path = os.path.join(dataset_directory, class_dir)
        image_files = [f for f in os.listdir(class_path) if f.endswith('.jpg')]

        if len(image_files) < min_samples_per_class:
            continue

        train_images, test_images = train_test_split(image_files, test_size=test_ratio, random_state=random_seed)

        for train_image in train_images:
            src_image_path = os.path.join(class_path, train_image)
            src_annotation_path = os.path.join(class_path, train_image.replace('.jpg', '.txt'))

            dest_image_path = os.path.join(train_directory, class_dir, train_image)
            dest_annotation_path = os.path.join(train_directory, class_dir, train_image.replace('.jpg', '.txt'))

            os.makedirs(os.path.dirname(dest_image_path), exist_ok=True)

            if os.path.exists(src_annotation_path):
                shutil.copy(src_image_path, dest_image_path)
                shutil.copy(src_annotation_path, dest_annotation_path)

        for test_image in test_images:
            src_image_path = os.path.join(class_path, test_image)
            src_annotation_path = os.path.join(class_path, test_image.replace('.jpg', '.txt'))

            dest_image_path = os.path.join(test_directory, class_dir, test_image)
            dest_annotation_path = os.path.join(test_directory, class_dir, test_image.replace('.jpg', '.txt'))

            os.makedirs(os.path.dirname(dest_image_path), exist_ok=True)

            if os.path.exists(src_annotation_path):
                shutil.copy(src_image_path, dest_image_path)
                shutil.copy(src_annotation_path, dest_annotation_path)

# Executing function for Data Splitting
dataset_directory = 'C:/Users/yongx/Desktop/Yong Xian/Bachelor of Information Systems (Data Analytics)/BSDA Semester 9/Capstone Project 2/BTDAC_Project/Dataset Repository/Brain Tumor labeled dataset'
custom_train_test_split(dataset_directory, test_ratio=0.2, random_seed=42, min_samples_per_class=2)

In [ ]:
# Define the augmentation pipeline with specified techniques
transform = A.Compose([
    A.Resize(640, 640),               # Resize images to 640x640
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),  # Normalize images (mean and std values can be adjusted)
    A.RandomRotate90(p=0.5),          # Randomly rotate 90 degrees
    A.Flip(p=0.5),                    # Randomly flip the images
    A.Transpose(p=0.5),               # Randomly transpose the images
    A.RandomBrightnessContrast(p=0.5),# Adjust brightness and contrast randomly
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5), # Add Gaussian noise
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def preprocess_and_augment(image_path, annotation_path, output_dir, transform):
    """
    Applies resizing, normalization, and augmentation to images and adjusts bounding boxes accordingly.
    Saves the processed images and adjusted annotations.
    """
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Read the bounding boxes from the annotation file
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    bboxes = []
    class_labels = []
    # Parsing bounding boxes in YOLO format [class, center_x, center_y, width, height]
    for line in lines:
        class_number, centre_x, centre_y, width, height = map(float, line.strip().split())
        bboxes.append([centre_x, centre_y, width, height])
        class_labels.append(int(class_number))

    # Apply augmentation and get the transformed image and bounding boxes
    augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    augmented_image = augmented['image']
    augmented_bboxes = augmented['bboxes']
    augmented_labels = augmented['class_labels']

    # Save the augmented image
    augmented_image_path = os.path.join(output_dir, os.path.basename(image_path).replace('.jpg', '_aug.jpg'))
    cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    # Save the adjusted bounding boxes
    augmented_annotation_path = augmented_image_path.replace('.jpg', '.txt')
    with open(augmented_annotation_path, 'w') as file:
        for bbox, label in zip(augmented_bboxes, augmented_labels):
            file.write(f"{label} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

# Define paths to dataset and output directory
output_directory = 'C:/Users/yongx/Desktop/Yong Xian/Bachelor of Information Systems (Data Analytics)/BSDA Semester 9/Capstone Project 2/BTDAC_Project/Dataset Repository/Augmented Dataset'
os.makedirs(output_directory, exist_ok=True)

# Iterate over each class subdirectory and augment images
for class_name in ['glioma', 'meningioma', 'pituitary', 'notumor']:  # Ensure these match your folder names
    class_directory = os.path.join(dataset_directory, class_name)
    image_files = [f for f in os.listdir(class_directory) if f.endswith('.jpg')]

    for image_file in image_files:
        image_path = os.path.join(class_directory, image_file)
        annotation_path = image_path.replace('.jpg', '.txt')

        # Ensure the annotation file exists before processing
        if os.path.exists(annotation_path):
            preprocess_and_augment(image_path, annotation_path, output_directory, transform)


In [ ]:
# Load the YOLOv8 model configuration (adjust model size: 'yolov8n.yaml', 'yolov8s.yaml', etc.)
model = YOLO('yolov8s.yaml')  # Choose the appropriate model variant

# Train the model using the data configuration file
model.train(data='C:/Users/yongx/Desktop/Yong Xian/Bachelor of Information Systems (Data Analytics)/BSDA Semester 9/Capstone Project 2/BTDAC_Project/brain_tumor_dataset.yaml', 
            epochs=50, 
            imgsz=640, 
            batch=16, 
            project='C:/Users/yongx/Desktop/Yong Xian/Bachelor of Information Systems (Data Analytics)/BSDA Semester 9/Capstone Project 2/BTDAC_Project/Best Model')  # Adjust epochs and batch size as needed

In [ ]:
app = Flask(__name__)
model = YOLO('path_to/best.pt')  # Load your trained model weights

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        file = request.files['file']
        image_path = os.path.join('uploads', file.filename)
        file.save(image_path)

        # Predict and draw bounding boxes
        results = model.predict(image_path, save=True, imgsz=640, conf=0.3)
        output_image_path = results[0].path  # Path to saved prediction image

        return send_file(output_image_path, mimetype='image/jpeg')  # Display the processed image

    return render_template('upload.html')  # Render a simple HTML upload form

if __name__ == '__main__':
    app.run(debug=True)